In [1]:
import os
import sys

TOP_DIR=os.path.realpath("../../..")
if TOP_DIR not in sys.path: sys.path.append(TOP_DIR)

In [2]:
import pandas as pd
import numpy as np
from lib.util.convert import standardise_columns, literal_converter
from lib.util.geo import normalise_leeds_wards, ward_name_to_code

In [3]:
data = pd.read_csv('../../../working/metrics/community/events.csv', parse_dates=['Start date'])
data = data.rename(columns=standardise_columns)

In [4]:
wards = data['ward_(from_venue)'].fillna('[]').pipe(literal_converter).apply(np.unique)

In [5]:
data['ward_name'] = wards.apply(lambda x: x[0] if len(x) > 0 else np.nan).pipe(normalise_leeds_wards)
data['ward_code'] = data.ward_name.pipe(ward_name_to_code)
data['audience'] = data.actual_audience_size_number_of_participants_in_person
cleaned_data = data.loc[:, ['event_name', 'project_name', 'start_date', 'ward_name', 'ward_code', 'audience']]

In [6]:
DATA_DIR='../../../data/metrics/community'
os.makedirs(DATA_DIR, exist_ok=True)
cleaned_data.to_csv(os.path.join(DATA_DIR, 'events.csv'), index=False)

Missing or duplicated wards

In [7]:
data.loc[wards.apply(len) != 1, ['project_name', 'event_name', 'start_date', 'ward_(from_venue)']]

,project_name,event_name,start_date,ward_(from_venue)
25,01 - The Awakening,Be More and Swarthmore Pack,2023-02-07,NaN
55,01 - The Awakening,Creation Station - Leeds City Council,2022-11-23,NaN
56,01 - The Awakening,Creation Station - Leeds City Council,2022-11-24,NaN
70,01 - The Awakening,Creation Station - Yorkshire Schools Dance Fes...,2022-11-19,NaN
72,01 - The Awakening,Creation Station: Ballot Engagement,NaT,NaN
...,...,...,...,...
297,Hidden Stories,Edible Archives: Eating The Corn Exchange Slot 2,2023-10-29,NaN
298,Hidden Stories,Harewood House - Aberford Community Cafe,2023-04-28,NaN
306,Hidden Stories,Lens Lab Exhibition,2023-09-16,NaN
312,Hidden Stories,Royal Armouries Exhibition,2023-09-07,NaN


Missing audience

In [8]:
data.loc[data.audience.isna(), ['project_name', 'event_name', 'start_date', 'audience']]

,project_name,event_name,start_date,audience
2,01 - The Awakening,Ballot Facilitated Session - Carr Manor Commun...,2022-11-01,NaN
3,01 - The Awakening,Ballot Facilitated Session - Chapel Allerton P...,2022-11-07,NaN
4,01 - The Awakening,Ballot Facilitated Session - Cobden Primary Sc...,2022-11-09,NaN
5,01 - The Awakening,Ballot Facilitated Session - Cobden Primary Sc...,2022-11-10,NaN
6,01 - The Awakening,Ballot Facilitated Session - Cobden Primary Sc...,2022-11-18,NaN
...,...,...,...,...
319,Hidden Stories,Try Your Hand at Ceramic Sculpture,2023-09-10,NaN
320,Hidden Stories,Try Your Hand at Ceramic Sculpture,2023-09-16,NaN
321,Hidden Stories,Try Your Hand at Stone Carving,2023-09-09,NaN
322,Hidden Stories,Try Your Hand at Stone Carving,2023-09-10,NaN
